<img src="https://raw.githubusercontent.com/OGGM/oggm/master/docs/_static/logo.png" width="40%"  align="left">

# Glacier dynamics and length changes

This code was used to make the analyses shown in this blog post: http://oggm.org/2017/10/23/length-changes/

Date: 25.10.2017, updated 02.02.2018

In [ ]:
import os
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
import oggm
from oggm import cfg, utils, tasks, workflow, graphics
from oggm.core.flowline import FileModel
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Plot directory
pdir = os.path.expanduser('~')

In [ ]:
cfg.initialize()
cfg.PATHS['working_dir'] = os.path.expanduser('~/OGGM_Example_Length')
cfg.PATHS['dem_file'] = utils.get_demo_file('srtm_oetztal.tif')
cfg.PATHS['climate_file'] = utils.get_demo_file('HISTALP_oetztal.nc')
cfg.PARAMS['run_mb_calibration'] = True
cfg.PARAMS['ys'] = 1855
cfg.PARAMS['ye'] = 2003  # This is the actual date of RGI
cfg.PARAMS['border'] = 160
cfg.PARAMS['use_intersects'] = False

In [ ]:
hef_file = utils.get_demo_file('Hintereisferner_RGI5.shp')
entity = gpd.GeoDataFrame.from_file(hef_file)

In [ ]:
gdir = workflow.init_glacier_regions(entity, reset=True)[0]
workflow.gis_prepro_tasks([gdir])
workflow.climate_tasks([gdir])
workflow.inversion_tasks([gdir])
tasks.init_present_time_glacier(gdir)

In [ ]:
# Observed length changes
df = gdir.get_ref_length_data()
df = df.loc[1855:2003]['dL']
df = df - df.iloc[-1]
df.plot();
plt.title('Observed length changes Hintereisferner (1855-2003 )');

## Random climates 

In [ ]:
odf = pd.DataFrame()
for seed in [0, 1, 2, 3, 4]:
    suff = 'rdn_mustar_seed{}'.format(seed)
    # bias=0 to ensure equibilbrium climate at t*. Needed for t* only!
    tasks.run_random_climate(gdir, bias=0, nyears=400, output_filesuffix=suff, seed=seed);
    ds = xr.open_dataset(gdir.get_filepath('model_diagnostics', filesuffix=suff))
    odf['Seed {}'.format(seed)] = ds.length_m
odf.index = ds.time
odf.index.name = 'Years'

In [ ]:
# 2003 length according to the model
len_0 = odf.iloc[0, 0]

In [ ]:
# Plot 1
# We do the running average over 36 months for nice looking plots but this isn't necessary 
ax = (odf.rolling(36, center=True).mean() - len_0).plot();
ax.set_ylabel('Glacier Length Change [m]');
ax.set_ylim([-1500, 1500])
plt.title('Hintereisferner length changes in a random climate')
plt.tight_layout();
plt.savefig(os.path.join(pdir, 'rdn_lengths.png'), dpi=150)

## Historical run from a current glacier 

In [ ]:
tasks.run_from_climate_data(gdir, output_filesuffix='hist_from_current');

In [ ]:
ds = xr.open_dataset(gdir.get_filepath('model_diagnostics', filesuffix='hist_from_current'))
s = ds.length_m.to_series().rolling(36, center=True).mean()
s = s - len_0
ax = df.plot(c='k', label='Observations');
s.plot(c='C0', label='OGGM');
plt.legend();
ax.set_ylabel('Glacier Length Change [m]');
plt.title('Hintereisferner length changes Experiment 2')
plt.tight_layout();
plt.savefig(os.path.join(pdir, 'lengths_from_today.png'), dpi=150)

## Historical run from a larger glacier

In [ ]:
suff = 'rdn_cold'
# bias=0 to ensure equibilbrium climate at t*. Needed for t* only!
tasks.run_random_climate(gdir, bias=0, nyears=400, temperature_bias=-0.5, output_filesuffix=suff, seed=3);

In [ ]:
ds = xr.open_dataset(gdir.get_filepath('model_diagnostics', filesuffix=suff))
s = ds.length_m.to_series().rolling(36, center=True).mean()
s = s - len_0
s.plot();
plt.title('Glacier length in a colder, random climate');

In [ ]:
tmp_mod = FileModel(gdir.get_filepath('model_run', filesuffix=suff))
tmp_mod.run_until(400)
graphics.plot_modeloutput_map(gdir, model=tmp_mod)

In [ ]:
tmp_mod = FileModel(gdir.get_filepath('model_run', filesuffix=suff))
tmp_mod.run_until(390)
tasks.run_from_climate_data(gdir, init_model_fls=tmp_mod.fls, output_filesuffix='hist_from_bias');

In [ ]:
ds = xr.open_dataset(gdir.get_filepath('model_diagnostics', filesuffix='hist_from_bias'))
s = ds.length_m.to_series().rolling(36, center=True).mean()
s = s - len_0
ax = df.plot(c='k', label='Observations');
s.plot(c='C0', label='OGGM');
plt.legend();
ax.set_ylabel('Glacier Length Change [m]');
plt.title('Hintereisferner length changes Experiment 3')
plt.tight_layout();
plt.savefig(os.path.join(pdir, 'lengths_from_colder_1.png'), dpi=150)

## Historical run from a liquid glacier

In [ ]:
suff = 'rdn_cold_a'
glen_a = cfg.A*2
tasks.run_random_climate(gdir, bias=0, nyears=400, temperature_bias=-0.5, output_filesuffix=suff, 
                         seed=3, glen_a=glen_a);

In [ ]:
ds = xr.open_dataset(gdir.get_filepath('model_diagnostics', filesuffix=suff))
s = ds.length_m.to_series().rolling(36, center=True).mean()
s = s - len_0
s.plot();
plt.title('Glacier length in a colder, random climate');

In [ ]:
tmp_mod = FileModel(gdir.get_filepath('model_run', filesuffix=suff))
tmp_mod.run_until(300)
tasks.run_from_climate_data(gdir, init_model_fls=tmp_mod.fls, output_filesuffix='hist_from_bias_a', glen_a=glen_a);

In [ ]:
ds = xr.open_dataset(gdir.get_filepath('model_diagnostics', filesuffix='hist_from_bias_a'))
s = ds.length_m.to_series().rolling(36, center=True).mean()
s = s - len_0
ax = df.plot(c='k', label='Observations');
s.plot(c='C0', label='OGGM');
plt.legend();
ax.set_ylabel('Glacier Length Change [m]');
plt.title('Hintereisferner length changes Experiment 4')
plt.tight_layout();
plt.savefig(os.path.join(pdir, 'lengths_from_colder_2.png'), dpi=150)

## How long are varying initial states visible?

We run the glacier with CESM Data for 500 years and different starting states

In [ ]:
# Process the data
f = utils.get_demo_file('cesm.TREFHT.160001-200512.selection.nc')
cfg.PATHS['gcm_temp_file'] = f
f = utils.get_demo_file('cesm.PRECC.160001-200512.selection.nc')
cfg.PATHS['gcm_precc_file'] = f
f = utils.get_demo_file('cesm.PRECL.160001-200512.selection.nc')
cfg.PATHS['gcm_precl_file'] = f
tasks.process_cesm_data(gdir)

In [ ]:
# First with default a
suff = 'rdn_cold'

In [ ]:
tmp_mod = FileModel(gdir.get_filepath('model_run', filesuffix=suff))
tmp_mod.run_until(0)
tasks.run_from_climate_data(gdir, ys=1601, ye=2003, init_model_fls=tmp_mod.fls,
                            climate_filename='cesm_data', output_filesuffix='cesm_from_bias');
tmp_mod.run_until(100)
tasks.run_from_climate_data(gdir, ys=1601, ye=2003, init_model_fls=tmp_mod.fls,
                            climate_filename='cesm_data', output_filesuffix='cesm_from_bias_100');
tmp_mod.run_until(200)
tasks.run_from_climate_data(gdir, ys=1601, ye=2003, init_model_fls=tmp_mod.fls,
                            climate_filename='cesm_data', output_filesuffix='cesm_from_bias_200');
tmp_mod.run_until(300)
tasks.run_from_climate_data(gdir, ys=1601, ye=2003, init_model_fls=tmp_mod.fls,
                            climate_filename='cesm_data', output_filesuffix='cesm_from_bias_300');

In [ ]:
pdf = pd.DataFrame()
ds = xr.open_dataset(gdir.get_filepath('model_diagnostics', filesuffix='cesm_from_bias'))
s = ds.length_m.to_series().rolling(36, center=True).mean()
pdf['Init HEF'] = s - len_0
ds = xr.open_dataset(gdir.get_filepath('model_diagnostics', filesuffix='cesm_from_bias_100'))
s = ds.length_m.to_series().rolling(36, center=True).mean()
pdf['Large HEF'] = s - len_0
ds = xr.open_dataset(gdir.get_filepath('model_diagnostics', filesuffix='cesm_from_bias_200'))
s = ds.length_m.to_series().rolling(36, center=True).mean()
pdf['Larger HEF'] = s - len_0
ds = xr.open_dataset(gdir.get_filepath('model_diagnostics', filesuffix='cesm_from_bias_300'))
s = ds.length_m.to_series().rolling(36, center=True).mean()
pdf['Even larger HEF'] = s - len_0

ax = pdf.plot();
df.plot(ax=ax, c='k', label='Observations');
plt.legend(loc='upper center');
ax.set_ylabel('Glacier Length Change [m]');
plt.title('Hintereisferner length changes CESM');
plt.tight_layout();
plt.savefig(os.path.join(pdir, 'lengths_from_cesm.png'), dpi=150)